In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datetime import datetime

# 데이터 분석 입문자를 위한 파이썬 판다스 300제
https://wikidocs.net/89378  

# DataManim - 빅데이터 분석기사 실기 문제 연습
https://www.datamanim.com/dataset/99_pandas/pandasMain.html

# 5. 데이터 프레임에 함수 적용하기

## 5-1. lambda 함수
- 이름없이 간단한 함수를 만드는 것 

In [2]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': [4, 5, 6, 7, 8], 'c': [10, 20, 30, 40, 50]})
df

,a,b,c
0,1,4,10
1,2,5,20
2,3,6,30
3,4,7,40
4,5,8,50


In [8]:
df['a'].map(lambda x: True if x % 2 == 0 else False)

0    False
1     True
2    False
3     True
4    False
Name: a, dtype: bool

In [7]:
df

,a,b,c,a-1
0,1,4,10,2
1,2,5,20,3
2,3,6,30,4
3,4,7,40,5
4,5,8,50,6


## 5-2. map() / apply() / applymap() 비교
- map() : 단일 컬럼(Series)에 적용
- apply() : DataFrame의 행 or 열에 적용
- applymap() : 개별 값에 적용

In [13]:
test = {1: '일', 2: '이'}

In [14]:
df['a'].map(test)

0      일
1      이
2    NaN
3    NaN
4    NaN
Name: a, dtype: object

In [15]:
df

,a,b,c,a-1
0,1,4,10,2
1,2,5,20,3
2,3,6,30,4
3,4,7,40,5
4,5,8,50,6


In [23]:
df.applymap(lambda x: x+10)

,a,b,c,a-1
0,11,14,20,12
1,12,15,30,13
2,13,16,40,14
3,14,17,50,15
4,15,18,60,16


In [24]:
df.loc[0, 'a'] = np.nan
df.loc[1, 'b'] = np.nan

In [31]:
df[['a', 'b']] = df[['a', 'b']]\
                .apply(lambda x: x.fillna(value=x.mean()))

In [26]:
df['a'].fillna(value=df['a'].mean())
df['b'].fillna(value=df['b'].mean())

0    4.00
1    6.25
2    6.00
3    7.00
4    8.00
Name: b, dtype: float64

# 연습문제
- dataset/titanic.csv 사용

In [32]:
df = pd.read_csv('dataset/titanic.csv')

In [34]:
df.head(2)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Q1. 성별을 'male'은 0으로, 'female'은 1로 변환하여 새로운 열 'gender_num'로 추가하세요.

In [36]:
df['gender'].unique()

array(['male', 'female'], dtype=object)

In [38]:
gender_map = {'male': 0, 'female': 1}
df['gender_num'] = df['gender'].map(gender_map)

In [40]:
df['gender'].map(lambda x: 0 if x == 'male' else 1)
np.where(df['gender'] == 'male', 0, 1) 

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    1
889    0
890    0
Name: gender, Length: 891, dtype: int64

## Q2. 탑승 항구 embarked를 'S'는 'Southampton', 'C'는 'Cherbourg', 'Q'는 'Queenstown'으로 매핑하여 새로운 열 'embarked_port'로 추가하세요.

In [42]:
df['embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [43]:
emb_map = {'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'}
df['embarked'].map(emb_map)

0      Southampton
1        Cherbourg
2      Southampton
3      Southampton
4      Southampton
          ...     
886    Southampton
887    Southampton
888    Southampton
889      Cherbourg
890     Queenstown
Name: embarked, Length: 891, dtype: object

In [44]:
def get_emp_port(x):
    if x == 'S':
        return 'Southampton'
    elif x == 'C':
        return 'Cherbourg'
    elif x == 'Q':
        return 'Queenstown'
    else:
        return None

In [45]:
df['embarked'].map(get_emp_port)

0      Southampton
1        Cherbourg
2      Southampton
3      Southampton
4      Southampton
          ...     
886    Southampton
887    Southampton
888    Southampton
889      Cherbourg
890     Queenstown
Name: embarked, Length: 891, dtype: object

## Q3. 이름의 길이가 20자 이상이면 '긴 이름', 10자 이상이면 '중간 이름', 그 외에는 '짧은 이름'을 반환하는 함수를 작성하고, 이를 'Name_Length_Class' 열에 적용하세요.

In [46]:
def get_length(name):
    if len(name) >= 20:
        return '긴이름'
    elif len(name) >= 10:
        return '중간 이름'
    else:
        return '짧은 이름'

In [48]:
df['Name_Length_Class'] = df['name'].map(get_length)

In [50]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,gender_num,Name_Length_Class
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,긴이름


## Q4. 선실 등급(Pclass)이 1인 승객의 운임 요금(fare)을 1.5배로 계산하고,  선실 등급이 2인 승객의 운임 요금은 해당 승객의 운임 요금을 1.2배로,  선실 등급이 3인 승객의 운임 요금은 그대로 유지하여  조정된 운임요금 열 'adj_fare'를 추가하세요.

In [54]:
def get_adj_fare(x):
    if x['pclass'] == 1:
        return x['fare'] * 1.5
    elif x['pclass'] == 2:
        return x['fare'] * 1.2
    else:
        return x['fare']

In [56]:
df['adj_fare'] = df[['pclass', 'fare']]\
                  .apply(lambda x: get_adj_fare(x), axis=1)

# 6. 날짜 데이터 처리
- 'dataset/AirQualityUCI.csv' 데이터 사용
- T: 온도
- RH: 상대습도
- AH: 절대 습도

In [89]:
df = pd.read_csv('dataset/AirQualityUCI.csv')

In [79]:
df['test_dt'] = '10/3/2004'

In [80]:
df.head(1)

,datetime,T,RH,AH,test_dt
0,2004-03-10 18:00:00,13.6,48.875001,0.757754,10/3/2004


In [83]:
df['test_dt'].str.split('/').str[-1].astype(int)

0       2004
1       2004
2       2004
3       2004
4       2004
        ... 
9352    2004
9353    2004
9354    2004
9355    2004
9356    2004
Name: test_dt, Length: 9357, dtype: int32

In [65]:
df['datetime'] = pd.to_datetime(df['datetime'])

## 6-1. 필요한 정보 추출하기
- dt.date
- dt time
---
- dt.year
- dt.month
- dt.day
- dt.hour
- dt.minute
- dt.second
---
- dt.weekday : The day of the week with Monday=0, Sunday=6.
- datetime.strptime(x, '%Y-%m-%d').strftime('%w') : Sunday=0, Saturday=6

In [67]:
df.head(1)

,datetime,T,RH,AH
0,2004-03-10 18:00:00,13.6,48.875001,0.757754


In [70]:
df['datetime'].dt.weekday

0       2
1       2
2       2
3       2
4       2
       ..
9352    0
9353    0
9354    0
9355    0
9356    0
Name: datetime, Length: 9357, dtype: int64

# map
# datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w') 활용해서 
# 'weekday' 컬럼 추가

In [90]:
df['datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w'))

0       3
1       3
2       3
3       3
4       3
       ..
9352    1
9353    1
9354    1
9355    1
9356    1
Name: datetime, Length: 9357, dtype: object

# diff, shift, rolling

In [91]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 5],
                   'b': [4, 5, 6, 7, 8],
                   'c': [10, 20, 30, 40, 50]})
df

,a,b,c
0,1,4,10
1,2,5,20
2,3,6,30
3,4,7,40
4,5,8,50


In [100]:
df['a'].rolling(3).sum()

0     NaN
1     NaN
2     6.0
3     9.0
4    12.0
Name: a, dtype: float64

# 7. 데이터 집계와 그룹연산
- dataset/diamonds.csv 사용

| 컬럼명      | 설명                                                         |
|-------------|--------------------------------------------------------------|
| carat       | 다이아몬드의 무게(carat)                                      |
| cut         | 다이아몬드의 귀빛(cut) 등급                                  |
| color       | 다이아몬드의 색상(color) 등급                                 |
| clarity     | 다이아몬드의 투명도(clarity) 등급                             |
| depth       | 다이아몬드의 심도(depth) 비율                                 |
| table       | 다이아몬드의 상단 테이블(table) 너비 비율                    |
| price       | 다이아몬드의 가격(price)                                      |
| x           | 다이아몬드의 가로 길이(x)                                     |
| y           | 다이아몬드의 세로 길이(y)                                     |
| z           | 다이아몬드의 깊이(z)                                           |
---
- groupby
- crosstab
- pivot_table
---

In [101]:
df = pd.read_csv('dataset/titanic.csv')
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


## 7-1. groupby

# pclass, survived별로 fare의 평균과 합계를 구하라

In [102]:
df.groupby(['pclass', 'survived'])['fare'].agg(['mean', 'sum'])

mean         sum
pclass survived                       
1      0         64.684008   5174.7206
       1         95.608029  13002.6919
2      0         19.412328   1882.9958
       1         22.055700   1918.8459
3      0         13.669364   5085.0035
       1         13.694887   1629.6916

## 7-2. crosstab

# 성별(gender)와 생존여부(survived)에 대한 교차표

In [113]:
pd.crosstab(index=df['gender'], columns=df['survived'])

survived,0,1
gender,,
female,81,233
male,468,109


In [112]:
df.groupby(['gender', 'survived']).size().unstack('survived')

survived,0,1
gender,,
female,81,233
male,468,109


## 7-3. pivot_table

# 클래스(pclass)와 생존여부(survived)별 운임(fare)의 평균을 구하라

In [115]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [116]:
pd.pivot_table(df,
               index='pclass',
               columns='survived',
               values='fare',
               aggfunc='mean')

survived,0,1
pclass,,
1,64.684008,95.608029
2,19.412328,22.055700
3,13.669364,13.694887


In [118]:
df.groupby(['pclass', 'survived'])['fare'].mean().unstack('survived')

survived,0,1
pclass,,
1,64.684008,95.608029
2,19.412328,22.055700
3,13.669364,13.694887


## 연습문제

In [119]:
df = pd.read_csv('dataset/diamonds.csv')

In [120]:
df.head(1)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43


# Q1.  다이아몬드의 cut 등급에 따른 평균 가격 계산하기

In [121]:
pd.pivot_table(df, index='cut', values='price', aggfunc='mean')

,price
cut,
Fair,2793.612903
Good,2185.303371
Ideal,2503.051051
Premium,2554.372414
Very Good,2365.309735


In [131]:
df.groupby('cut')['price'].mean()

cut
Fair         2793.612903
Good         2185.303371
Ideal        2503.051051
Premium      2554.372414
Very Good    2365.309735
Name: price, dtype: float64

## Q2. 다이아몬드의 color 등급과 clarity 등급에 따른 빈도표 만들기

In [132]:
pd.crosstab(index=df['color'], columns=df['clarity'])

clarity,I1,IF,SI1,SI2,VS1,VS2,VVS1,VVS2
color,,,,,,,,
D,1,0,60,22,9,23,7,7
E,4,3,87,25,33,65,9,14
F,7,3,44,33,58,59,13,9
G,3,6,28,20,26,34,5,17
H,6,1,39,27,15,15,14,8
I,6,1,30,16,13,13,10,6
J,2,0,18,11,5,9,0,1


In [135]:
df.groupby(['color', 'clarity']).size().unstack('clarity').fillna(value=0)

clarity,I1,IF,SI1,SI2,VS1,VS2,VVS1,VVS2
color,,,,,,,,
D,1.0,0.0,60.0,22.0,9.0,23.0,7.0,7.0
E,4.0,3.0,87.0,25.0,33.0,65.0,9.0,14.0
F,7.0,3.0,44.0,33.0,58.0,59.0,13.0,9.0
G,3.0,6.0,28.0,20.0,26.0,34.0,5.0,17.0
H,6.0,1.0,39.0,27.0,15.0,15.0,14.0,8.0
I,6.0,1.0,30.0,16.0,13.0,13.0,10.0,6.0
J,2.0,0.0,18.0,11.0,5.0,9.0,0.0,1.0


## Q3. 다이아몬드의 cut 등급에 따른 carat 평균과 price 중간값 계산하기

In [138]:
df.groupby('cut').agg({'carat': 'mean', 'price': 'median'})

,carat,price
cut,,
Fair,0.900806,2843.5
Good,0.645281,2801.0
Ideal,0.666306,2822.0
Premium,0.709828,2822.0
Very Good,0.656062,2809.5


In [140]:
df.groupby('cut')[['carat', 'price']].apply(lambda x: pd.Series({
    'carat': x['carat'].mean(),
    'price': x['price'].median()
}))

,carat,price
cut,,
Fair,0.900806,2843.5
Good,0.645281,2801.0
Ideal,0.666306,2822.0
Premium,0.709828,2822.0
Very Good,0.656062,2809.5


# 8. 그룹별 함수적용

In [142]:
df = pd.read_csv('dataset/titanic.csv')

In [143]:
df = df[['gender', 'fare']].head()

In [144]:
df

,gender,fare
0,male,7.2500
1,female,71.2833
2,female,7.9250
3,female,53.1000
4,male,8.0500


# 성별(gender)별로 요금(fare)의 평균값을 가지는 새로운 컬럼 'fare_mean'을 만드세요

## 8-1. apply

In [147]:
df_mean = df.groupby('gender')['fare'].mean()
df_mean

gender
female    44.102767
male       7.650000
Name: fare, dtype: float64

In [150]:
pd.merge(df,
         df_mean,
         left_on='gender',
         right_index=True,
         how='left',
         suffixes=('', '_mean'))

,gender,fare,fare_mean
0,male,7.2500,7.650000
1,female,71.2833,44.102767
2,female,7.9250,44.102767
3,female,53.1000,44.102767
4,male,8.0500,7.650000


## 8-2. transform()

In [152]:
df['fare_mean2'] = df.groupby('gender')['fare'].transform(lambda x: x.mean())

In [153]:
df

,gender,fare,fare_mean2
0,male,7.2500,7.650000
1,female,71.2833,44.102767
2,female,7.9250,44.102767
3,female,53.1000,44.102767
4,male,8.0500,7.650000


# 1. age의 결측치를 pclass별 age 중간값으로 대체하세요.

In [154]:
df = pd.read_csv('dataset/titanic.csv')

In [158]:
df['age'] = df.groupby('pclass')['age']\
              .transform(lambda x: x.fillna(value=x.median()))

In [159]:
df['age'].isna().sum()

0

# 2. pclass별 탑승자수를 new_column으로 정의하세요.(기존 데이터프레임 인덱스 유지) 탑승자 수는 고유한 passengerId를 기준으로 한다

In [164]:
df['new_column'] = df.groupby('pclass')['passengerId']\
                     .transform(lambda x: x.nunique())

# 연습문제
- dataset/titanic.csv 활용

## Q1. titanic 데이터셋에서 각 승객 등급('pclass')별로 생존한 승객 수를 계산하고, 이를 원래 데이터프레임에 새로운 열 'SurvivedCountByClass'로 추가하세요.

## Q2. titanic 데이터셋에서 age의 결측치를 그 승객의 성별의 평균나이로 대체하세요.